In [1]:
import gymnasium as gym
import ale_py
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import imageio
from tqdm import tqdm
from torch.distributions import Categorical
import matplotlib.pyplot as plt
import imageio

In [2]:
class ActorCriticNetwork(nn.Module):
    def __init__(self, input_shape, output_dim):
        super(ActorCriticNetwork, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
        )
        
        self.policy_head = nn.Linear(512, output_dim)
        self.value_head = nn.Linear(512, 1)

    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self, x):
        x = self.conv(x).view(x.size()[0], -1)
        x = self.fc(x)
        policy = self.policy_head(x)
        value = self.value_head(x)
        return policy, value

In [5]:
class PPOAgent:
    def __init__(self, env, gamma, lr, epsilon, batch_size):
        self.env = env
        self.gamma = gamma
        self.lr = lr
        self.epsilon = epsilon
        self.batch_size = batch_size

        self.state_shape = (4, 84, 84)
        self.num_actions = env.action_space.n

        # Модель сети
        self.network = ActorCriticNetwork(self.state_shape, self.num_actions).to(device)
        self.optimizer = optim.Adam(self.network.parameters(), lr=self.lr)

        self.states = []
        self.actions = []
        self.log_probs = []
        self.values = []
        self.rewards = []
        self.dones = []
        self.returns = []  # Для хранения вознаграждений после обновления
        self.advantages = []  # Для хранения преимуществ

    def act(self, state):
        state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(device)
        policy, value = self.network(state)

        dist = Categorical(logits=policy)
        action = dist.sample()

        self.log_probs.append(dist.log_prob(action))
        self.values.append(value)

        self.states.append(state)  
        self.actions.append(action)

        return action.item()

    def store_transition(self, reward, done):
        self.rewards.append(reward)
        self.dones.append(done)

    def compute_advantages(self, next_value):
        advantages = []
        gae = 0
        for t in reversed(range(len(self.rewards))):
            delta = self.rewards[t] + self.gamma * (self.dones[t] * next_value - self.values[t]) - self.values[t]
            gae = delta + self.gamma * gae
            advantages.insert(0, gae)
            next_value = self.values[t]
        return advantages

    def compute_returns(self):
        returns = []
        return_ = 0
        for reward, done in zip(reversed(self.rewards), reversed(self.dones)):
            return_ = reward + self.gamma * return_ * (1 - done)
            returns.insert(0, return_)
        return returns

    def update(self, states, actions, log_probs, values, rewards, dones):
        if len(rewards) == 0:
            raise ValueError("Rewards list is empty, unable to compute returns or advantages.")

        states = torch.cat([torch.tensor(state, dtype=torch.float32) for state in states]).to(device)
        actions = torch.tensor(actions).to(device)
        old_log_probs = torch.stack(log_probs).detach()

        returns = self.compute_returns()  
        
        if len(returns) == 0:
            raise ValueError("Returns list is empty, unable to compute advantages.")

        # Вычисляем преимущества
        advantages = self.compute_advantages(returns[-1])

        # Конвертируем returns и advantages в тензоры
        returns = torch.tensor(returns).to(device)
        advantages = torch.tensor(advantages).to(device)

        # Получаем прогнозы и ценности из сети
        policy, value = self.network(states)
        dist = Categorical(logits=policy)
        log_probs = dist.log_prob(actions)

        # Вычисление соотношения и потерь
        ratio = torch.exp(log_probs - old_log_probs)
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1 - self.epsilon, 1 + self.epsilon) * advantages

        policy_loss = -torch.min(surr1, surr2).mean()
        value_loss = (returns - value).pow(2).mean()

        loss = policy_loss + 0.5 * value_loss

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.log_probs = []
        self.values = []
        self.rewards = []
        self.dones = []
        self.states = []  # Очищаем состояния
        self.actions = []  # Очищаем действия


In [7]:
def show_frame(frame):
    plt.imshow(frame)
    plt.axis('off')
    plt.show()

In [9]:
def train(env, device, episodes, batch_size, gamma, epsilon, lr):
    agent = PPOAgent(env, gamma, lr, epsilon, batch_size)

    for episode in tqdm(range(episodes)):
        state, _ = env.reset()
        state_gray = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY)
        state_resized = cv2.resize(state_gray, (84, 84))
        state_stack = np.stack([state_resized] * 4, axis=0)

        done = False
        episode_reward = 0

        while not done:
            action = agent.act(state_stack)
            next_state, reward, done, truncated, _ = env.step(action)

            next_state_gray = cv2.cvtColor(next_state, cv2.COLOR_RGB2GRAY)
            next_state_resized = cv2.resize(next_state_gray, (84, 84))
            next_state_stack = np.append(state_stack[1:], np.expand_dims(next_state_resized, axis=0), axis=0)

            agent.store_transition(reward, done)

            state_stack = next_state_stack
            episode_reward += reward

        agent.update(agent.states, agent.actions, agent.log_probs, agent.values, agent.rewards, agent.dones)

        if episode % 100 == 0:
            print(f"Episode {episode}, Reward: {episode_reward}")
            # show_frame(next_state)
            
        print(f"End of Episode {episode}: Total Reward: {episode_reward}")

    torch.save(agent.network.state_dict(), "/Users/romain/Rein/PPO/battlezone_ppo_model.pth")
    print("Model saved as battlezone_ppo_model1.pth")

In [11]:
if __name__ == "__main__":
    env = gym.make('ALE/BattleZone-v5', render_mode='rgb_array')  # Инициализация игры
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    episodes = 3000
    batch_size = 64
    gamma = 0.99
    epsilon = 0.1  
    lr = 0.01

    train(
        env, 
        device, 
        episodes, 
        batch_size, 
        gamma, 
        epsilon, 
        lr
    )

A.L.E: Arcade Learning Environment (version 0.10.1+6a7e0ae)
[Powered by Stella]
  0%|                                                  | 0/3000 [00:00<?, ?it/s]/var/folders/2q/07z6y2z90rq677n461j3mbxr0000gn/T/ipykernel_40113/3604788393.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states = torch.cat([torch.tensor(state, dtype=torch.float32) for state in states]).to(device)
  0%|                                        | 1/3000 [00:02<1:58:25,  2.37s/it]

Episode 0, Reward: 0.0
End of Episode 0: Total Reward: 0.0


  0%|                                        | 2/3000 [00:04<2:00:12,  2.41s/it]

End of Episode 1: Total Reward: 0.0


  0%|                                        | 3/3000 [00:07<2:00:37,  2.41s/it]

...

End of Episode 2997: Total Reward: 0.0


100%|████████████████████████████████████▉| 2999/3000 [2:31:33<00:03,  3.05s/it]

End of Episode 2998: Total Reward: 0.0


100%|█████████████████████████████████████| 3000/3000 [2:31:36<00:00,  3.03s/it]

End of Episode 2999: Total Reward: 0.0
Model saved as battlezone_ppo_model1.pth


In [ ]:
def test():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    env = gym.make('ALE/BattleZone-v5', render_mode="human")
    
    state_shape = (4, 84, 84)  # размер стека состояний (4 кадра)
    num_actions = env.action_space.n  # ко

    # Создаем агента PPO
    agent = PPOAgent(
        env, 
        gamma=0.99, 
        lr=0.00025, 
        epsilon=0.2,  
        batch_size=64
    )

    agent.network.load_state_dict(torch.load("/Users/romain/Rein/PPO/battlezone_ppo_model.pth", map_location=device))
    agent.network.eval() 
    agent.epsilon = 0.05  

    for episode in range(10):
        state, _ = env.reset()

        state_gray = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY)
        state_resized = cv2.resize(state_gray, (84, 84))
        
        # Стек состояний (4 кадра)
        state_stack = np.stack([state_resized] * 4, axis=0)
        
        total_reward = 0
        done = False

        while not done:
            action = agent.act(state_stack)

            next_state, reward, done, truncated, _ = env.step(action)
            
            # Преобразование следующего состояния
            next_state_gray = cv2.cvtColor(next_state, cv2.COLOR_RGB2GRAY)
            next_state_resized = cv2.resize(next_state_gray, (84, 84))
            
            # Обновляем стек состояний
            next_state_stack = np.append(state_stack[1:], np.expand_dims(next_state_resized, axis=0), axis=0)
            
            state_stack = next_state_stack  
            total_reward += reward

        print(f"Episode {episode + 1}: Total Reward: {total_reward}")

    env.close()

if __name__ == "__main__":
    test()

2024-12-13 08:20:45.571 python[40113:2433987] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-13 08:20:45.571 python[40113:2433987] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Episode 1: Total Reward: 0.0
